In [1]:
import sys

### yield

In [25]:
def infinite_sequence():
    num = 0
    while True:
        yield num
        num += 1
        yield f"this is the {num}th iteration"

gen = infinite_sequence()

In [32]:
value = next(gen)
print(value, type(value))

3 <class 'int'>


### List Comprehension vs Generator Comprehension

In [2]:
num_squared_lc = [num ** 2 for num in range(1000000)]
num_squared_gc = (num ** 2 for num in range(1000000))

#### **Memory usage comparison**

In [3]:
sys.getsizeof(num_squared_lc)

8448728

In [4]:
sys.getsizeof(num_squared_gc)

200

The Generator uses considerable less amount of memory

#### **Speed comparison**

In [ ]:
import cProfile

cProfile.run("sum([num ** 2 for num in range(1000000)])")

         334 function calls (327 primitive calls) in 0.043 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.006    0.006 _base.py:337(_invoke_callbacks)
        1    0.000    0.000    0.006    0.006 _base.py:537(set_result)
        1    0.000    0.000    0.037    0.037 asyncio.py:200(_handle_events)
        2    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
        2    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        1    0.000    0.000    0.000    0.000 base_events.py:1879(_add_callback)
        2    0.005    0.003    0.062    0.031 base_events.py:1894(_run_once)
        4    0.000    0.000    0.000    0.000 base_events.py:1989(get_debug)
        2    0.000    0.000    0.000    0.000 base_

In [6]:
cProfile.run("sum((num ** 2 for num in range(1000000)))")

         1000538 function calls (1000527 primitive calls) in 0.113 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen abc>:121(__subclasscheck__)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1390(_handle_fromlist)
  1000001    0.059    0.000    0.059    0.000 <string>:1(<genexpr>)
        1    0.000    0.000    0.039    0.039 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 _base.py:337(_invoke_callbacks)
        1    0.000    0.000    0.000    0.000 _base.py:537(set_result)
        1    0.002    0.002    0.067    0.067 asyncio.py:200(_handle_events)
        1    0.000    0.000    0.000    0.000 asyncio.py:225(add_callback)
        5    0.000    0.000    0.000    0.000 attrsettr.py:43(__getattr__)
        5    0.000    0.000    0.000    0.000 attrsettr.py:66(_get_attr_opt)
        1    0.000    0.000    0.000    0.000 base_ev

#### **StopIteration**

StopIteration is a natural exception that’s raised to signal the end of an iterator. For loops, for example, are built around StopIteration. You can even implement your own for loop by using a while loop:

In [7]:
letters = ["a", "b", "c", "d"]
letters_it = iter(letters)

while True:
    try:
        letter = next(letters_it)
    except StopIteration:
        break
    print(letter)



a
b
c
d


#### **How to Use .send()**

In [38]:
def is_palindrome(num):
    if num // 10 == 0:
        return False
    temp = num
    reversed_num = 0

    while temp != 0:
        reversed_num = (reversed_num * 10) + (temp % 10)
        temp = temp // 10
    
    if num == reversed_num:
        return True
    else: 
        return False

def infinite_palindromes():
    num = 0
    while True:
        if is_palindrome(num):
            i = (yield num)
            if i is not None:
                num = i
        num += 1

is_palindrome(101)

True

In [39]:
# pal_gen = infinite_palindromes()
# for i in pal_gen:
#     digits = len(str(i))
#     pal_gen.send(10 ** (digits))

### **How to use .throw()**

.throw() allows you to throw exceptions with the generator. In the below example, you raise the exception in line 6. This code will throw a ValueError once digits reaches 5:

In [42]:
pal_gen = infinite_palindromes()

for i in pal_gen:
    print(i)
    digits = len(str(i))
    if digits == 5:
        pal_gen.throw(ValueError("No more than 5 digits"))
    pal_gen.send(10 ** (digits))

11
111
1111
10101


ValueError: No more than 5 digits

### **How to Use .close()**
As its name implies, .close() allows you to stop a generator. This can be especially handy when controlling an infinite sequence generator. Let’s update the code above by changing .throw() to .close() to stop the iteration:

In [45]:
pal_gen = infinite_palindromes()
for i in pal_gen:
    print(i)
    digits = len(str(i))
    if digits == 5:
        pal_gen.close()
    pal_gen.send(10 ** (digits))

11
111
1111
10101


StopIteration: 

Instead of calling .throw(), you use .close() in line 6. The advantage of using .close() is that it raises StopIteration, an exception used to signal the end of a finite iterator:

### **Creating Data Pipelines with Generators**

Data pipelines allow you to string together code to process large datasets or streams of data without maxing out your machine’s memory.

Let’s think of a strategy:

Read every line of the file.
Split each line into a list of values.
Extract the column names.
Use the column names and lists to create a dictionary.
Filter out the rounds you aren’t interested in.
Calculate the total interest values for the year 2024.


In [ ]:
file_name = "data/saldos.csv"
lines = (line for line in open(file_name)) # generator type
list_line = (s.rstrip().split(",") for s in lines)
cols = next(list_line)
row_dicts = (dict(zip(cols, row)) for row in list_line)

interes_year = (
    float(item_dict["interes"])
    for item_dict in row_dicts 
    if int(item_dict["mes"][-4:]) == 2024
)

total_interes = sum(interes_year)
print(f"Total de interes de anio 2024 en USD($): {total_interes}")

Total de interes de anio 2024 en USD($): 341.62


In [90]:
cols

['mes', 'saldo anterior', 'ingreso', 'retiros', 'ahorro', 'interes']

In [91]:
next(row_dicts)

StopIteration: 

In [79]:
"06/2023"[-4:]

'2023'

##### *Important Note*
*The methods for handling CSV files developed in this tutorial are important for understanding how to use generators and the Python yield statement. However, when you work with CSV files in Python, you should instead use the csv module included in Python’s standard library. This module can parse comma-separated files more robustly and has optimized methods for handling them efficiently*

#### References

[Real Python: How to Use Generators and yield in Python](https://realpython.com/introduction-to-python-generators/#how-to-use-close)